In [2]:
%load_ext autoreload
%autoreload 2

In [21]:
import os, sys, ast
import ujson as json
from tqdm import tqdm
import random
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:90% !important; }</style>"))
pd.options.display.max_colwidth = 500
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5000)

### Load Squad + Entity Predictions

A current task we're exploring now is how to add entity embeddings from our system Bootleg [https://github.com/HazyResearch/bootleg] into downstream tasks. This is a simple model we trained that adds Bootleg entity embeddings to Squad (like, the _very_ first attempt we've done at this). The goal is to do some error analysis to understand if the entity embeddings are helping and, importantly, what can we do to improve performance

In [66]:
dump = Path("predictions_dump_filtered_to_incorrect_preds.csv")
df = pd.read_csv(dump)
# Due to csv saving arrays as strings, we need to do some conversions ... this is HACKY
df["gold_answer"] = df["gold_answer"].apply(lambda x: ast.literal_eval(x.replace("\n", "").replace("' '", "', '")))
df["ent_mentions"] = df["ent_mentions"].apply(lambda x: ast.literal_eval(x.replace("\n", "").replace("' '", "', '")))
df["ent_qids"] = df["ent_qids"].apply(lambda x: eval(x.replace("\n", "").replace("' '", "', '")))
df["ent_qids_with_unk"] = df["ent_qids_with_unk"].apply(lambda x: eval(x.replace("\n", "").replace("' '", "', '")))
df["ent_spans"] = df["ent_spans"].apply(lambda x: ast.literal_eval(x.replace("\n", "").replace("array", ",").replace("[,", "[")))
df["gold_answer_start"] = df["gold_answer_start"].apply(lambda x: eval(" ".join(x.split()).replace("[ ", "[").replace(" ]", "]").replace(" ", ", ")))

### Step 1: Look at a few examples
The goal is just to see what's going on. As you can see, it's hard to parse all the text. At the moment, I kind of just deal with it, but I'll show some statistical stuff later on.

In [67]:
df.sample(30)

,f1_score,input_text,prediction,gold_answer,ent_mentions,ent_qids,proportion_toks_as_ents,question,context,ent_qids_with_unk,ent_spans,example_id,gold_answer_start
1749,80.000000,"[CLS] in which galleries are the french paintings donated by jones displayed? [SEP] several french paintings entered the collection as part of the 260 paintings and miniatures ( not all the works were french, for example carlo crivelli's virgin and child ) that formed part of the jones bequest of 1882 and as such are displayed in the galleries of continental art 1600 – 1800, including the portrait of francois, duc d'alencon by francois clouet, gaspard dughet and works by francois boucher inc...",continental art,"[continental art 1600–1800, of continental art 1600–1800, galleries of continental art]","[jones, miniatures, virgin and child ), jones, bequest, portrait, francois,, francois clouet,, gaspard dughet, francois boucher, portrait, madame de pompadour, jean francois de troy,, jean - baptiste]","[Q204943, Q282129, Q926743, Q216406, Q211557, Q134307, Q180932, Q336747, Q741375, Q180932, Q134307, Q188965, Q707729, Q347139]",0.101562,In which galleries are the French paintings donated by Jones displayed?,"Several French paintings entered the collection as part of the 260 paintings and miniatures (not all the works were French, for example Carlo Crivelli's Virgin and Child) that formed part of the Jones bequest of 1882 and as such are displayed in the galleries of continental art 1600–1800, including the portrait of François, Duc d'Alençon by François Clouet, Gaspard Dughet and works by François Boucher including his portrait of Madame de Pompadour dated 1758, Jean François de Troy, Jean-Bapti...","[UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, Q204943, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, Q282129, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, Q926743, Q926743, Q926743, Q926743, UNK, UNK, UNK, UNK, UNK, Q216406, Q211557, Q211557, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, Q134307, UNK, Q180932, Q180932, UNK, UNK, UNK, UNK, UNK, UNK, UNK, Q336747, Q336747,...","[[10, 11], [27, 28], [44, 48], [53, 54], [54, 56], [75, 76], [77, 79], [86, 91], [91, 95], [98, 101], [103, 104], [105, 111], [114, 119], [119, 122]]",5726f755708984140094d737,"[263, 260, 250]"
1686,80.000000,"[CLS] what does phycoerytherin appear in? [SEP] phycobilins are a third group of pigments found in cyanobacteria, and glaucophyte, red algal, and cryptophyte chloroplasts. phycobilins come in all colors, though phycoerytherin is one of the pigments that makes many red algae red. phycobilins often organize into relatively large protein complexes about 40 nanometers across called phycobilisomes. like photosystem i and atp synthase, phycobilisomes jut into the stroma, preventing thylakoid stack...",red algae red,"[red algae, red algae, algae]","[pigments, cyanobacteria,, algal,, chloroplasts., colors,, pigments, algae, protein complexes, nanometers, atp, jut, stroma,, chloroplasts., chloroplasts, cyanobacteria, pigments]","[Q910979, Q93315, Q37868, Q47263, Q1075, Q161179, Q37868, Q420927, Q178674, Q80863, Q154845, Q557179, Q47263, Q47263, Q93315, Q910979]",0.135417,What does phycoerytherin appear in?,"Phycobilins are a third group of pigments found in cyanobacteria, and glaucophyte, red algal, and cryptophyte chloroplasts. Phycobilins come in all colors, though phycoerytherin is one of the pigments that makes many red algae red. Phycobilins often organize into relatively large protein complexes about 40 nanometers across called phycobilisomes. Like photosystem I and ATP synthase, phycobilisomes jut into the stroma, preventing thylakoid stacking in red algal chloroplasts. Cryptophyte chlor...","[UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, Q910979, Q910979, UNK, UNK, Q93315, Q93315, Q93315, Q93

### Step 2: Gather some simple aggregates

In [74]:
average_f1 = df["f1_score"].mean()
max_f1 = df["f1_score"].max()

# We'll approximate if a QID was in the question by looking at the first 7 tokens (a UNK means no entity)
df["is_ent_in_question"] = df.apply(lambda x: not all([a == "UNK" for a in x["ent_qids_with_unk"][:7]]), axis=1)

average_num_ents = df["proportion_toks_as_ents"].mean()
max_num_ents = df["proportion_toks_as_ents"].max()
num_early_ent_qs = df["is_ent_in_question"].sum()
# Bin f1 score for group by
df["f1_score_bin"] = df["f1_score"].apply(lambda x: int(x/10)*10)

grp_by = df[["f1_score_bin", "proportion_toks_as_ents", "is_ent_in_question"]].groupby(['f1_score_bin']).mean()

print(f"Average F1 {average_f1}, Max F1 {max_f1}")
print(f"Average Num Ents {average_num_ents}, Max Num Ents {max_num_ents}")
print(f"Num Early Entity Questions {num_early_ent_qs}")
display(grp_by)

Average F1 39.16742963701258, Max F1 96.7741935483871
Average Num Ents 0.1166794654797726, Max Num Ents 0.5963541666666666
Num Early Entity Questions 1073


,proportion_toks_as_ents,is_ent_in_question
f1_score_bin,,
0,0.119593,0.553731
10,0.104026,0.554054
20,0.111824,0.544554
30,0.120525,0.535211
40,0.115291,0.491228
50,0.111199,0.531646
60,0.120515,0.509259
70,0.116102,0.595238
80,0.116995,0.548638


There are other metrics I'd consider computing
* A distribution over the QIDs that appear in the questions? Are some more popular than others? Does this correlate with performance?
* Breaking down performance by the length of the sentence/question
* Where is the answer with respect to the question?

### Step 3: Add aditional metadata

I often have metadata associate with an entity that may be of interest, and I'd like to add that to my analysis. I'm not going to show this as it's the same as above except looking at accuracy over the metadata.

For example, suppose I have a mapping of QID types -> the type of that entity. (I normally have this in a json). I would then get the predicted type of the entities. I could then compute our f1 score with respect to the majority type in the sentence (or something like that)

### Step 4: Share
The final step is to share the results with others. I typically plot or just simply print tables and add them to a power point or even slack conversation.